In [19]:
#importando librerias necesarias
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import re
from selenium import webdriver
import time
import json

## Localbitcoin Ventas

In [4]:
i = 0
r = []
#se realiza un ciclo para obtener las primeras 3 paginas del api
while i < 3:
    url = 'https://localbitcoins.com/sell-bitcoins-online/ves/.json?page='+ str(i)
    r.append( requests.get(url).json())
    #se aplica un sleep para evitar que el api bloquee la conexion debido multiples request simultaneos
    time.sleep(10)
    i += 1

In [20]:
# se crea una nueva lista para guardar los valores aplanados
dflist=[]
i = 0
#se realiza un ciclo por cada pagina obtenida de la api
while i < 3:
    # el json del api devuelve una lista de ofertas
    #vamos a obtener cada oferta y guardarlo en un dataframe
    data = json.dumps(r[i]['data']['ad_list'])
    data2 = json.loads(data)
    df = pd.DataFrame(data2)
#print(df['data'])
    #se obtienen los valores de las listas internas en json de las columnas
    df1 = pd.DataFrame(df['data'].values.tolist())
    df2 = pd.DataFrame(df1['profile'].values.tolist())
#display (df2)
    # se guarda los valores en una lista, eliminando los valores originales que aplanamos
    dflist.append( pd.concat([df1.drop('profile', axis=1), df2], axis=1) )
    i += 1

In [21]:
#se crea un dataframe para limpieza y
df1 = pd.DataFrame(df['data'].values.tolist())
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
#display (df1)

In [23]:
#se verifica el tamaño de registros obtenidos por la pagina
#al realizar la ejecucion continuamente se nota que los anuncios tienen un tiempo determinado
# por lo que cada 15 min se obtienen nuevos anuncios si se ejecuta el proceso
dfsell = pd.concat(dflist, ignore_index=True)
#numero de registros
len(dfsell)

117

In [24]:
#Limpieza de atributos en la columna de banco
#La idea es normalizar los nombre si se encuentra alguna frase que haga mencion a los bancos mas importantes en venezuela
dfsell.loc[(dfsell['bank_name'].str.contains("mercantil|Mercantil|MERCANTIL"), 'bank_name' )] = 'MERCANTIL'
dfsell.loc[(dfsell['bank_name'].str.contains("banesco|Banesco|BANESCO"), 'bank_name' )] = 'BANESCO'
dfsell.loc[(dfsell['bank_name'].str.contains("BOD|bod|Bod"), 'bank_name' )] = 'BOD'
dfsell.loc[(dfsell['bank_name'].str.contains("VENEZUELA|venezuela|Venezuela|BDV"), 'bank_name' )] = 'VENEZUELA'
dfsell.loc[(dfsell['bank_name'].str.contains("provincial|Provincial|PROVINCIAL"), 'bank_name' )] = 'PROVINCIAL'
dfsell.loc[(dfsell['bank_name'].str.contains("BVC|bvc|Venezolano|VENEZOLANO"), 'bank_name' )] = 'BVC'
dfsell.loc[(dfsell['bank_name'].str.contains("BNC|bnc|Credito|credito|Crédito|crédito|CREDITO|CRÉDITO"), 'bank_name' )] = 'BNC'
dfsell.loc[(dfsell['bank_name'].str.contains("bicentenario|Bicentenario|BICENTENARIO"), 'bank_name' )] = 'BICENTENARIO'
dfsell.loc[(dfsell['bank_name'].str.contains("Tesoro|tesoro|TESORO"), 'bank_name' )] = 'TESORO'
dfsell.loc[(dfsell['bank_name'].str.contains("Caribe|caribe|CARIBE|bancaribe|BANCARIBE"), 'bank_name' )] = 'BANCARIBE'
dfsell.loc[(dfsell['bank_name'].str.contains("FONDO|Fondo|fondo|BFC"), 'bank_name' )] = 'FONDOCOMUN'
dfsell.loc[(dfsell['bank_name'].str.contains("banplus|Banplus|BANPLUS"), 'bank_name' )] = 'BANPLUS'
dfsell.loc[(dfsell['bank_name'].str.contains("petro|Petro|PETRO"), 'bank_name' )] = 'PETRO'
dfsell.loc[(dfsell['bank_name'].str.contains("movil|Movil|MOVIL|movíl|Movíl|MOVÍL"), 'bank_name' )] = 'PAGOMOVIL'
dfsell = dfsell.replace('\n',' ', regex=True)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
#display (dfcomplete)
#dfcomplete.groupby('bank_name').size()

In [25]:
#se eliminan las columnas sin relevancia para el analisis
del dfsell['require_feedback_score']
del dfsell['hidden_by_opening_hours']
del dfsell['first_time_limit_btc']
del dfsell['require_trusted_by_advertiser']
del dfsell['visible']
del dfsell['is_low_risk']
del dfsell['atm_model']
del dfsell['name']
#display (dfcomplete)


In [27]:
# dataframe resultante con las ofertas de venta del dia
dfsell.head()

,location_string,countrycode,city,trade_type,online_provider,volume_coefficient_btc,sms_verification_required,currency,lat,lon,min_amount,max_amount,max_amount_available,ad_id,temp_price_usd,temp_price,created_at,require_trade_volume,msg,bank_name,require_identification,is_local_office,payment_window_minutes,limit_to_fiat_amounts,username,trade_count,feedback_score,last_online
0,Venezuela,VE,,ONLINE_BUY,SPECIFIC_BANK,1.50,True,VES,0.0,0.0,100000000.00,850000000.00,850000000.00,1331471,39306.15,64000000000.00,2021-02-06T22:11:17+00:00,0.0,Transferencias rapidas y seguras,VENEZUELA,False,False,90,None,Davidson07,4,100,2021-02-11T04:58:14+00:00
1,"Caracas, Capital District, Venezuela",VE,Caracas,ONLINE_BUY,SPECIFIC_BANK,1.50,False,VES,0.0,0.0,20000000.00,950000000.00,950000000.00,1284375,38930.52,63388384873.51,2020-11-09T23:30:11+00:00,0.0,compramos bitcoins por BOD pagamos en 5 minuto...,BOD,False,False,90,None,cryptozuelans,500+,100,2021-02-11T14:14:15+00:00
2,Venezuela,VE,,ONLINE_BUY,SPECIFIC_BANK,1.50,False,VES,0.0,0.0,5000.00,1000354000.00,1000354000.00,1308969,37980.99,61842326705.86,2020-12-29T07:09:42+00:00,0.0,Transferencias desde Banco VENEZUELA Si demoro...,VENEZUELA,False,False,90,None,hrbg,7,100,2021-02-01T17:59:38+00:00
3,Venezuela,VE,,ONLINE_BUY,SPECIFIC_BANK,1.50,False,VES,0.0,0.0,1000000.00,196840000.00,196840000.00,1317015,35651.90,58050000000.00,2021-01-11T05:32:26+00:00,0.0,Favor enviar coordenadas bancarias sin puntos ...,BICENTENARIO,False,False,90,None,emilys_96,100+,100,2021-02-02T20:03:41+00:00
4,Venezuela,VE,,ONLINE_BUY,NATIONAL_BANK,1.50,False,VES,0.0,0.0,2000000.00,958383148.00,958383148.00,1331005,35132.42,57204152202.92,2021-02-06T04:34:54+00:00,0.0,**Solo pago móvil** CI 21413880 04125405412 B...,⚡Pago Móvil ⚡,False,False,90,None,rogersdesantis,5,100,2021-02-07T05:22:11+00:00


## Localbitcoin Compra

In [28]:
# Se aplica el mismo proceso pero en este caso se trabajara con las ofertas ce compra
i = 0
r = []
#se realiza un ciclo para obtener las primeras 3 paginas del api
while i < 3:
    url = 'https://localbitcoins.com/buy-bitcoins-online/ves/.json?page='+ str(i)
    r.append( requests.get(url).json())
    #se aplica un sleep para evitar que el api bloquee la conexion debido multiples request simultaneos
    time.sleep(10)
    i += 1

In [29]:
dflist=[]
i = 0
while i < 3:
    data = json.dumps(r[i]['data']['ad_list'])
    data2 = json.loads(data)
    df = pd.DataFrame(data2)
#print(df['data'])
    df1 = pd.DataFrame(df['data'].values.tolist())
    df2 = pd.DataFrame(df1['profile'].values.tolist())
#display (df2)
    dflist.append( pd.concat([df1.drop('profile', axis=1), df2], axis=1) )
    i += 1

In [30]:
df1 = pd.DataFrame(df['data'].values.tolist())
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
#display (df1)

In [31]:
dfbuy = pd.concat(dflist, ignore_index=True)
#longitud de registros
len(dfbuy)

134

In [32]:
#Se aplica el mismo tipo de limpieza del nombre de las entidades bancarias
dfbuy.loc[(dfbuy['bank_name'].str.contains("mercantil|Mercantil|MERCANTIL"), 'bank_name' )] = 'MERCANTIL'
dfbuy.loc[(dfbuy['bank_name'].str.contains("banesco|Banesco|BANESCO"), 'bank_name' )] = 'BANESCO'
dfbuy.loc[(dfbuy['bank_name'].str.contains("BOD|bod|Bod"), 'bank_name' )] = 'BOD'
dfbuy.loc[(dfbuy['bank_name'].str.contains("VENEZUELA|venezuela|Venezuela|BDV"), 'bank_name' )] = 'VENEZUELA'
dfbuy.loc[(dfbuy['bank_name'].str.contains("provincial|Provincial|PROVINCIAL"), 'bank_name' )] = 'PROVINCIAL'
dfbuy.loc[(dfbuy['bank_name'].str.contains("BVC|bvc|Venezolano|VENEZOLANO"), 'bank_name' )] = 'BVC'
dfbuy.loc[(dfbuy['bank_name'].str.contains("BNC|bnc|Credito|credito|Crédito|crédito|CREDITO|CRÉDITO"), 'bank_name' )] = 'BNC'
dfbuy.loc[(dfbuy['bank_name'].str.contains("bicentenario|Bicentenario|BICENTENARIO"), 'bank_name' )] = 'BICENTENARIO'
dfbuy.loc[(dfbuy['bank_name'].str.contains("Tesoro|tesoro|TESORO"), 'bank_name' )] = 'TESORO'
dfbuy.loc[(dfbuy['bank_name'].str.contains("Caribe|caribe|CARIBE|bancaribe|BANCARIBE"), 'bank_name' )] = 'BANCARIBE'
dfbuy.loc[(dfbuy['bank_name'].str.contains("FONDO|Fondo|fondo|BFC"), 'bank_name' )] = 'FONDOCOMUN'
dfbuy.loc[(dfbuy['bank_name'].str.contains("banplus|Banplus|BANPLUS"), 'bank_name' )] = 'BANPLUS'
dfbuy.loc[(dfbuy['bank_name'].str.contains("petro|Petro|PETRO"), 'bank_name' )] = 'PETRO'
dfbuy.loc[(dfbuy['bank_name'].str.contains("movil|Movil|MOVIL|movíl|Movíl|MOVÍL"), 'bank_name' )] = 'PAGOMOVIL'
dfbuy = dfbuy.replace('\n',' ', regex=True)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
#display (dfcomplete)
#dfcomplete.groupby('bank_name').size()

In [33]:
#se eliminan la misma cantidad de columnas para que  concuerden con el anterior dataframe y permita el merge
del dfbuy['require_feedback_score']
del dfbuy['hidden_by_opening_hours']
del dfbuy['first_time_limit_btc']
del dfbuy['require_trusted_by_advertiser']
del dfbuy['visible']
#del dfcomplete['is_low_risk']
del dfbuy['atm_model']
del dfbuy['name']
#display (dfcomplete)

In [34]:
#dataframe con ofertas de compra
dfbuy.head()

,location_string,countrycode,city,trade_type,online_provider,volume_coefficient_btc,sms_verification_required,currency,lat,lon,min_amount,max_amount,max_amount_available,ad_id,temp_price_usd,temp_price,created_at,require_trade_volume,msg,bank_name,require_identification,is_local_office,payment_window_minutes,limit_to_fiat_amounts,username,trade_count,feedback_score,last_online
0,Venezuela,VE,,ONLINE_SELL,SPECIFIC_BANK,1.50,True,VES,0.0,0.0,20000000.00,8000000000.00,8000000000.00,932550,58338.30,94989000000.00,2019-02-06T16:34:20+00:00,0.0,Se solicitara documentación del usuario y titu...,MERCANTIL,False,False,90,,juancho17,15 000+,100,2021-02-11T14:13:15+00:00
1,Venezuela,VE,,ONLINE_SELL,SPECIFIC_BANK,1.50,True,VES,0.0,0.0,10000000.00,8000000000.00,8000000000.00,932548,58344.45,94999000000.00,2019-02-06T16:33:51+00:00,0.0,Se solicitara documentación del usuario y titu...,BANCARIBE,False,False,90,,juancho17,15 000+,100,2021-02-11T14:13:15+00:00
2,Venezuela,VE,,ONLINE_SELL,SPECIFIC_BANK,1.50,True,VES,0.0,0.0,50000000.00,8000000000.00,8000000000.00,932551,58344.45,94999000000.00,2019-02-06T16:34:23+00:00,0.0,Se solicitara documentación del usuario y titu...,PROVINCIAL,False,False,90,,juancho17,15 000+,100,2021-02-11T14:13:15+00:00
3,"Caracas, Capital District, Venezuela",VE,Caracas,ONLINE_SELL,SPECIFIC_BANK,1.50,False,VES,0.0,0.0,100000000.00,2000000000.00,1325280529.10,848910,58959.22,96000000080.00,2018-09-30T00:46:36+00:00,0.0,✅SI ES USUARIO NUEVO!! DEBE ENVIAR DE FORMA IN...,BANCARIBE,True,False,90,,Indestructible2018,3000+,100,2021-02-11T14:04:15+00:00
4,Venezuela,VE,,ONLINE_SELL,SPECIFIC_BANK,0.00,False,VES,0.0,0.0,1000000.00,1974600000.00,1974600000.00,1303026,59345.30,96628635477.91,2020-12-16T18:47:20+00:00,0.0,Activo 24H Hasta 150.000.000 - 150M Bs Valid...,MERCANTIL,False,False,90,None,angeldoza25,1000+,100,2021-02-11T14:24:15+00:00


In [35]:
#concatenacion de los dataframe de anuncios de venta y compra
#dataframe general de la pagina
dflocalbitcoin= pd.concat([dfsell, dfbuy])

In [36]:
#añadiendo como columna la fuente de los datos
dflocalbitcoin['source'] = 'LocalBitcoin'
dflocalbitcoin['coin']= 'bitcoin'
# dataframe general de la pagina
dflocalbitcoin = dflocalbitcoin[['username','trade_count','bank_name','location_string','trade_type','temp_price','source','coin']]
dflocalbitcoin.head()

,username,trade_count,bank_name,location_string,trade_type,temp_price,source,coin
0,Davidson07,4,VENEZUELA,Venezuela,ONLINE_BUY,64000000000.00,LocalBitcoin,bitcoin
1,cryptozuelans,500+,BOD,"Caracas, Capital District, Venezuela",ONLINE_BUY,63388384873.51,LocalBitcoin,bitcoin
2,hrbg,7,VENEZUELA,Venezuela,ONLINE_BUY,61842326705.86,LocalBitcoin,bitcoin
3,emilys_96,100+,BICENTENARIO,Venezuela,ONLINE_BUY,58050000000.00,LocalBitcoin,bitcoin
4,rogersdesantis,5,⚡Pago Móvil ⚡,Venezuela,ONLINE_BUY,57204152202.92,LocalBitcoin,bitcoin


## Localcryptos compra

In [37]:
#obtener las ofertas de compra
url ='https://localcryptos.com/Bitcoin/Venezuela/Buy'
html = requests.get(url).content

In [38]:
soup = BeautifulSoup(html, "lxml")
# Buscar la informacion por el tag y clase
elem = [elem.text if 'Venezuela' in elem.text else None for elem in soup.find_all('div', {'class':'_HChx0m _2UfjJ2'})]
#limpieza de valores null
elem = list(filter(None,elem))
#limpieza de campos y colocacion de | como separador
elem = [ re.sub(r'No trades yet', r'||' ,i) for i in elem  ]
elem = [ re.sub(r'~(.*)trades', r'|\1|' ,i) for i in elem  ]
elem = [ re.sub(r' (.*)trades', r'|\1|' ,i) for i in elem  ]
elem = [ re.sub('"Venezuela', '|Venezuela ' ,i) for i in elem  ]
elem = [ re.sub(r'Caracas', r'Caracas|' ,i) for i in elem  ]
elem = [ re.sub(r'Maracay', r'Maracay|' ,i) for i in elem  ]
elem = [ re.sub(r'Porlamar', r'Porlamar|' ,i) for i in elem  ]
elem = [ re.sub(r'Táchira', r'Táchira|' ,i) for i in elem  ]
elem = [ re.sub(r'Cristóbal', r'Cristóbal| ' ,i) for i in elem  ]
elem = [ re.sub(r'Barinas', r'Barinas|' ,i) for i in elem  ]
elem = [ re.sub(r'Guasipati', r'Guasipati|' ,i) for i in elem  ]
elem = [ re.sub(r'Barquisimeto', r'Barquisimeto|' ,i) for i in elem  ]
elem = [ re.sub(r'Buy', r'|Buy' ,i) for i in elem  ]
#elem = [ re.sub(r'Bs', r'|Bs' ,i) for i in elem  ]
#elem = [ re.sub(r'$', r'|$' ,i) for i in elem  ]
#elem = [ re.sub(r'€', r'|€' ,i) for i in elem  ]
elem = [ i.split('|') for i in elem  ]
#elem

In [39]:
#columns=[['user','cant_trades','metodo_pago','']]
dfbuy = pd.DataFrame(elem)
del dfbuy[6]
#renombre de columnas y agregacion de columnas predefinadas
dfbuy.columns =['username','trade_count','bank_name','location_string','temp_price','trade_type']
dfbuy['trade_type']= 'ONLY_BUY'
dfbuy['source']= 'localcryptos'
dfbuy['coin']= 'bitcoin'

## Localcryptos venta

In [40]:
#obtener las ofertas de Venta
url ='https://localcryptos.com/Bitcoin/Venezuela/Sell'
html = requests.get(url).content

In [41]:
soup = BeautifulSoup(html, "lxml")
# Buscar la informacion por el tag y clase
elem = [elem.text if 'Venezuela' in elem.text else None for elem in soup.find_all('div', {'class':'_HChx0m _2UfjJ2'})]
#limpieza de valores null
elem = list(filter(None,elem))
#limpieza de campos y colocacion de | como separador
elem = [ re.sub(r'No trades yet', r'||' ,i) for i in elem  ]
elem = [ re.sub(r'~(.*)trades', r'|\1|' ,i) for i in elem  ]
elem = [ re.sub(r' (.*)trades', r'|\1|' ,i) for i in elem  ]
elem = [ re.sub('"Venezuela', '|Venezuela ' ,i) for i in elem  ]
elem = [ re.sub(r'Caracas', r'Caracas|' ,i) for i in elem  ]
elem = [ re.sub(r'Maracay', r'Maracay|' ,i) for i in elem  ]
elem = [ re.sub(r'Porlamar', r'Porlamar|' ,i) for i in elem  ]
elem = [ re.sub(r'Táchira', r'Táchira|' ,i) for i in elem  ]
elem = [ re.sub(r'Cristóbal', r'Cristóbal| ' ,i) for i in elem  ]
elem = [ re.sub(r'Barinas', r'Barinas|' ,i) for i in elem  ]
elem = [ re.sub(r'Guasipati', r'Guasipati|' ,i) for i in elem  ]
elem = [ re.sub(r'Barquisimeto', r'Barquisimeto|' ,i) for i in elem  ]
elem = [ re.sub(r'Sell', r'|Sell' ,i) for i in elem  ]
#elem = [ re.sub(r'Bs', r'|Bs' ,i) for i in elem  ]
#elem = [ re.sub(r'$', r'|$' ,i) for i in elem  ]
#elem = [ re.sub(r'€', r'|€' ,i) for i in elem  ]
elem = [ i.split('|') for i in elem  ]
#elem

In [42]:
#columns=[['user','cant_trades','metodo_pago','']]
dfsell = pd.DataFrame(elem)
#del dfsell[6]
#renombre de columnas y agregacion de columnas predefinadas
dfsell.columns =['username','trade_count','bank_name','location_string','temp_price','trade_type']
dfsell['trade_type']= 'ONLY_SELL'
dfsell['source']= 'localcryptos'
dfsell['coin']= 'bitcoin'

In [43]:
# Dataframe global de la pagina
dflocalcryptos= pd.concat([dfsell, dfbuy])

In [44]:
dflocalcryptos.head()

,username,trade_count,bank_name,location_string,temp_price,trade_type,source,coin
0,DolaresEnVzla,"1,100","*Cash deposit""Pago Movil - cualquier banco",Venezuela San Antonio del Táchira,"Bs.S 750K to 5,50MBs.S 3,63B4% below market",ONLY_SELL,localcryptos,bitcoin
1,churonfeng,95,"*Bank transfer""🟢Banesco",Venezuela Caracas,"Bs.S 30M to 1BBs.S 3,63B4% below market",ONLY_SELL,localcryptos,bitcoin
2,jeff0102,190,"*Bank transfer""Mercantil,BDV,Provincial,bansc",Venezuela Caracas,"Bs.S 500K to 250MBs.S 3,25B14% below market",ONLY_SELL,localcryptos,bitcoin
3,andersernesto,"1,700","*Bank transfer""BANESCO",Venezuela Caracas,"Bs.S 70M to 500MBs.S 2,64B30% below market",ONLY_SELL,localcryptos,bitcoin
4,instacristhian,"20,000","*Bank transfer""BANESCO PROVINCIAL MERCANTIL",Venezuela Maracay,"Bs.S 10M to 150MBs.S 1,30B66% below market",ONLY_SELL,localcryptos,bitcoin


## Binance buy

In [66]:
#para ocultar el navegador
#options = webdriver.FirefoxOptions()
#options.headless = True
#browser = webdriver.Firefox(options=options)
browser = webdriver.Firefox()
browser.get('https://c2c.binance.com/es/trade/buy/BTC')
time.sleep(5)

In [67]:
#funciona para obtener una valor usando selenium a partir de una clave en css
def obtener_valor(browser,css):
    lists= browser.find_elements_by_css_selector(css)
    list_r = []
    for l in lists:
        list_r.append(l.text)
    return list_r

In [68]:
#funciona para obtener una valor usando selenium a partir de una tag id
def obtener_valor_id(browser,ids):
    lists= browser.find_elements_by_id(ids)
    list_r = []
    for l in lists:
        list_r.append(l.text)
    return list_r

In [69]:
#funciona para obtener los bancos
def obtener_binance_banco(browser,x,y):
    lists_caja= browser.find_elements_by_css_selector("div[class='css-1w5q6ud']")
    i =0
    list_banco = []
    for l in lists_caja:
        lists= l.find_elements_by_css_selector("div[class='css-uj9ajp'][data-bn-type='text']")
        s = ''
        for listitem in lists:
            s += listitem.get_attribute('title')+', '
        list_banco.append(s)
    return list_banco   

In [70]:
# funcion para verificar si nos encontramos en la ultima pagina de las ofertas
def ultimapagina(browser):
    try:
        browser.find_element_by_css_selector("button[aria-label='Next page'][disabled='']")
    except NoSuchElementException:
        #verificar que este entrando en la excepcion
        #print('siguiente pagina')
        return True
    return False

#### cambiar a moneda VES

In [71]:
#selecionar el boton para cambio de moneda
time.sleep(2)
button = browser.find_element_by_id('C2Cfiatfilter_searhbox_fiat')
button.click()

In [72]:
#ingresar VES para cambiar de moneda
time.sleep(2)
menu1 = browser.find_element_by_css_selector("input[placeholder='']")
menu1.send_keys('VES', Keys.ENTER)

### Extraccion con varias paginas

In [73]:
#bloque de codigo con todas las busquedas que se realiza en 1 pagina, para que se ejecute automaticamente
# este bloque genera varias listas con la informacion de todas las paginas de oferta de compra
list_user = []
list_trade = []
list_banco = []
list_price = []
c = 0
siguiente = True
while siguiente:
    list_user+= obtener_valor_id(browser,'C2Cofferlistsell_link_merchant')
    list_trade+= obtener_valor(browser,"div[class='css-1a0u4z7']")
    list_banco+= obtener_binance_banco(browser,"div[class='css-1w5q6ud']","div[class='css-uj9ajp'][data-bn-type='text']")
    list_price+= obtener_valor(browser, "div[class='css-1m1f8hn']" )
    button = browser.find_element_by_css_selector("button[aria-label='Next page']")
    button.click()
    if ultimapagina(browser) == False and c == 0:
        c = 1
    elif ultimapagina(browser) == False and c == 1:
        siguiente = False
    else:
        pass
    time.sleep(5)

#### username (1 pagina)

In [ ]:
list_user = obtener_valor_id(browser,'C2Cofferlistsell_link_merchant')

In [ ]:
list_user

#### trade_count (1 pagina)

In [ ]:
list_trade= obtener_valor(browser,"div[class='css-1a0u4z7']")

In [ ]:
list_trade

#### bank_name (1 pagina)

In [ ]:
list_banco= obtener_binance_banco(browser,"div[class='css-1w5q6ud']","div[class='css-uj9ajp'][data-bn-type='text']")

In [ ]:
list_banco

### temp_price (1 pagina)

In [ ]:
list_price = obtener_valor(browser, "div[class='css-1m1f8hn']" )

In [ ]:
list_price

### concatenacion de todas las listas

In [75]:
#concatenacion de todas las listas para generar el dataframe
dfbuy = pd.DataFrame(list(zip(list_user, list_trade, list_banco , list_price)), 
               columns =['username', 'trade_count', 'bank_name', 'temp_price' ])
#creacion de columnas generadas por defecto
dfbuy['location_string'] = 'Venezuela'
dfbuy['trade_type']= 'ONLY_BUY'
dfbuy['source']= 'Binance'
dfbuy['coin']= 'bitcoin'
#dataframe general de compra de binance
dfbuy = dfbuy[['username', 'trade_count', 'bank_name','location_string','trade_type', 'temp_price','source','coin']]
dfbuy.head()

,username,trade_count,bank_name,location_string,trade_type,temp_price,source,coin
0,AbrahamR,216 órdenes,"Pago Movil, Provincial,",Venezuela,ONLY_BUY,"80,441,070,732.90",Binance,bitcoin
1,Briggsup,1036 órdenes,"Banesco, Pago Movil,",Venezuela,ONLY_BUY,"80,569,134,420.12",Binance,bitcoin
2,✔️CriptoCambiosVzla⭐,437 órdenes,"Banesco, Mercantil, Provincial,",Venezuela,ONLY_BUY,"80,833,620,784.46",Binance,bitcoin
3,LEINITH,134 órdenes,"Banesco, Pago Movil,",Venezuela,ONLY_BUY,"81,192,377,697.92",Binance,bitcoin
4,allyzc,1852 órdenes,"Banesco, Transferencia bancaria (Venezuela), P...",Venezuela,ONLY_BUY,"81,704,632,446.81",Binance,bitcoin


## Binance sell

In [76]:
#mover selenium a la pagina de ofertas de venta
browser.get('https://c2c.binance.com/es/trade/sell/BTC')
time.sleep(5)

#### cambiar a moneda VES

In [77]:
#mismo procedimiento para cambiar el tipo de moneda
time.sleep(2)
button = browser.find_element_by_id('C2Cfiatfilter_searhbox_fiat')
button.click()

In [78]:
time.sleep(2)
menu1 = browser.find_element_by_css_selector("input[placeholder='']")
menu1.send_keys('VES', Keys.ENTER)

### Extraccion con varias paginas

In [79]:
#bloque de codigo con todas las busquedas que se realiza en 1 pagina, para que se ejecute automaticamente
# este bloque genera varias listas con la informacion de todas las paginas de oferta de venta
list_user = []
list_trade = []
list_banco = []
list_price = []
c = 0
siguiente = True
while siguiente:
    list_user+= obtener_valor_id(browser,"C2Cofferlistbuy_link_merchantdetail")
    list_trade+= obtener_valor(browser,"div[class='css-1a0u4z7']")
    list_banco+= obtener_binance_banco(browser,"div[class='css-1w5q6ud']","div[class='css-uj9ajp'][data-bn-type='text']")
    list_price+= obtener_valor(browser, "div[class='css-1m1f8hn']" )
    button = browser.find_element_by_css_selector("button[aria-label='Next page']")
    button.click()
    if ultimapagina(browser) == False and c == 0:
        c = 1
    elif ultimapagina(browser) == False and c == 1:
        siguiente = False
    else:
        pass
    time.sleep(5)

#### username (1 pagina)

In [ ]:
list_user= obtener_valor_id(browser,"C2Cofferlistbuy_link_merchantdetail")

In [ ]:
list_user

#### trade_count (1 pagina)

In [ ]:
list_trade= obtener_valor(browser,"div[class='css-1a0u4z7']")

In [ ]:
list_trade

#### bank_name (1 pagina)

In [ ]:
list_banco = obtener_binance_banco(browser,"div[class='css-1w5q6ud']","div[class='css-uj9ajp'][data-bn-type='text']")

In [ ]:
list_banco

### temp_price (1 pagina)

In [ ]:
list_price = obtener_valor(browser, "div[class='css-1m1f8hn']" )

In [ ]:
list_price

### concatenacion de todas las listas

In [81]:
#concatenacion de todas las listas para generar el dataframe
dfsell = pd.DataFrame(list(zip(list_user, list_trade, list_banco , list_price)), 
               columns =['username', 'trade_count', 'bank_name', 'temp_price' ])
#creacion de columnas generadas por defecto
dfsell['location_string'] = 'Venezuela'
dfsell['trade_type']= 'ONLY_Sell'
dfsell['source']= 'Binance'
dfsell['coin']= 'bitcoin'
#dataframe general de ventas de binance
dfsell = dfsell[['username', 'trade_count', 'bank_name','location_string','trade_type', 'temp_price','source','coin']]
dfsell.head()

,username,trade_count,bank_name,location_string,trade_type,temp_price,source,coin
0,fülle2021latam,3 órdenes,"Transferencia Bancaria, Banesco, Transferencia...",Venezuela,ONLY_Sell,"80,130,101,791.89",Binance,bitcoin
1,SinFrontera,1303 órdenes,"Banesco, Provincial,",Venezuela,ONLY_Sell,"77,500,000,000.00",Binance,bitcoin
2,NelsonP23,398 órdenes,"Banesco,",Venezuela,ONLY_Sell,"77,352,258,263.10",Binance,bitcoin
3,Jose paredez,733 órdenes,"Banesco,",Venezuela,ONLY_Sell,"76,941,992,141.93",Binance,bitcoin
4,Jose paredez,733 órdenes,"Mercantil,",Venezuela,ONLY_Sell,"76,839,425,611.63",Binance,bitcoin


#### concatenar ambos dataframe para crear el general

In [82]:
dfbinance= pd.concat([dfsell, dfbuy])
dfbinance.head()

,username,trade_count,bank_name,location_string,trade_type,temp_price,source,coin
0,fülle2021latam,3 órdenes,"Transferencia Bancaria, Banesco, Transferencia...",Venezuela,ONLY_Sell,"80,130,101,791.89",Binance,bitcoin
1,SinFrontera,1303 órdenes,"Banesco, Provincial,",Venezuela,ONLY_Sell,"77,500,000,000.00",Binance,bitcoin
2,NelsonP23,398 órdenes,"Banesco,",Venezuela,ONLY_Sell,"77,352,258,263.10",Binance,bitcoin
3,Jose paredez,733 órdenes,"Banesco,",Venezuela,ONLY_Sell,"76,941,992,141.93",Binance,bitcoin
4,Jose paredez,733 órdenes,"Mercantil,",Venezuela,ONLY_Sell,"76,839,425,611.63",Binance,bitcoin


In [83]:
#cerrrar el navegador
browser.close()

## Dataframe merge de las 3 fuentes

In [84]:
dftotal= pd.concat([dflocalbitcoin, dflocalcryptos, dfbinance])
dftotal= dftotal.reset_index(drop=True)

In [85]:
dftotal.head()

,username,trade_count,bank_name,location_string,trade_type,temp_price,source,coin
0,Davidson07,4,VENEZUELA,Venezuela,ONLINE_BUY,64000000000.00,LocalBitcoin,bitcoin
1,cryptozuelans,500+,BOD,"Caracas, Capital District, Venezuela",ONLINE_BUY,63388384873.51,LocalBitcoin,bitcoin
2,hrbg,7,VENEZUELA,Venezuela,ONLINE_BUY,61842326705.86,LocalBitcoin,bitcoin
3,emilys_96,100+,BICENTENARIO,Venezuela,ONLINE_BUY,58050000000.00,LocalBitcoin,bitcoin
4,rogersdesantis,5,⚡Pago Móvil ⚡,Venezuela,ONLINE_BUY,57204152202.92,LocalBitcoin,bitcoin


## Export a CSV

In [86]:
#exportar en CSV con | como separador
dftotal.to_csv('cripto.csv', encoding='utf-8', index=False, sep='|')